In [1]:
import pandas as pd

# Function to parse the time windows column
def parse_time_windows(tw_str):
    tws = tw_str.split(';')  # Split multiple time windows
    time_windows = []
    for tw in tws:
        start, end = tw.split(',')
        time_windows.append((int(start), int(end)))
    return time_windows

# Reading the file and parsing the data
def read_instance_file(file_path):
    # Read the file into a pandas dataframe
    df = pd.read_csv(file_path, delimiter='\t')

    # Parse the time windows
    df['TWS'] = df['TWS'].apply(parse_time_windows)

    return df

# Example usage
file_path = 'SVRP-SPM-master/RD01N10.txt'  # Replace with your file path
data = read_instance_file(file_path)



#set column "CUST NO." equal to the index
for i in range(len(data)):
    data.at[i, 'CUST NO.'] = i

# Display the dataframe
print(data)


    CUST NO.   XCOORD.    YCOORD.  DEMAND  TWNUM.  \
0        0.0  30.52211  114.36467       0       1   
1        1.0  30.47700  114.29200       1       1   
2        2.0  30.49000  114.41700       1       3   
3        3.0  30.47300  114.32000       3       2   
4        4.0  30.60100  114.35000       2       1   
5        5.0  30.50200  114.38800       2       2   
6        6.0  30.61600  114.39200       1       2   
7        7.0  30.60800  114.36100       2       3   
8        8.0  30.52200  114.33600       1       1   
9        9.0  30.49800  114.39600       1       2   
10      10.0  30.48000  114.32100       1       1   
11      11.0  30.61100  114.38500       3       1   

                                    TWS  
0                            [(0, 480)]  
1                          [(178, 243)]  
2   [(45, 105), (113, 174), (188, 250)]  
3               [(29, 206), (223, 400)]  
4                          [(318, 434)]  
5               [(48, 166), (194, 313)]  
6               

In [ ]:
import copy
import pandas as pd

class Job:
    def __init__(
        self,
        cust_no=0,
        xcoord=0.0,
        ycoord=0.0,
        demand=0,   
        duration=0,
        time_windows=None,
    ):
        self.cust_no = cust_no
        self.xcoord = xcoord
        self.ycoord = ycoord
        self.demand = demand
        self.duration = duration
        self.time_windows = time_windows if time_windows is not None else []

    def __repr__(self):
        return (
            f"Job(cust_no={self.cust_no}, xcoord={self.xcoord}, ycoord={self.ycoord}, "
            f"demand={self.demand}, time_windows={self.time_windows})"
        )


def parse_time_windows(tw_str):
    # Parse the time windows string into a list of tuples
    if isinstance(tw_str, str):
        tws = tw_str.split(';')
        time_windows = []
        for tw in tws:
            if ',' in tw:
                start, end = tw.split(',')
                time_windows.append((int(start), int(end)))
            else:
                time_windows.append((int(tw), int(tw)))  # Handle single values
        return time_windows
    else:
        return []  # Handle cases where the TWS value is NaN or not a string


def read_jobs_from_instance_file(file_path):
    # Define the columns expected in the file
    #columns = ['XCOORD', 'YCOORD', 'DEMAND', 'TWNUM', 'TWS']
    
    # Read the file into a dataframe
    df = pd.read_csv(file_path, delimiter='\t')
    for i in range(len(df)):
        df.at[i, 'CUST NO.'] = i
    # List to store all jobs
    jobs = []
    print(df)

    # Iterate over the dataframe rows and create Job objects
    for _, row in df.iterrows():
        time_windows = parse_time_windows(row['TWS'])
        job = Job(
            cust_no=int(row['CUST NO.']),
            xcoord=row['XCOORD.'],
            ycoord=row['YCOORD.'],
            demand=row['DEMAND'],
            duration=1,
            time_windows=time_windows
        )
        jobs.append(job)

    # create copy of job 0 and append it to the end of the list
    job0 = copy.deepcopy(jobs[0])
    job0.cust_no = len(jobs)
    jobs.append(job0)

    return jobs


# Example usage
file_path = 'instance.txt'  # Replace with your file path
jobs = read_jobs_from_instance_file(file_path)

# Display the list of jobs
for job in jobs:
    print(job)

# create the distance matrix
import numpy as np
import math

def euclidean_distance(x1, y1, x2, y2):
    return math.sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2)

def create_distance_matrix(jobs):
    n = len(jobs)
    distance_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            if i != j:
                distance_matrix[i, j] = euclidean_distance(
                    jobs[i].xcoord, jobs[i].ycoord, jobs[j].xcoord, jobs[j].ycoord
                )
    return distance_matrix

# Create the distance matrix
distance_matrix = create_distance_matrix(jobs)
print(distance_matrix)

    CUST NO.   XCOORD.    YCOORD.  DEMAND  TWNUM.                     TWS
0        0.0  30.52211  114.36467       0       1                   0,480
1        1.0  30.47700  114.29200       1       1                 178,243
2        2.0  30.49000  114.41700       1       3  45,105;113,174;188,250
3        3.0  30.47300  114.32000       3       2          29,206;223,400
4        4.0  30.60100  114.35000       2       1                 318,434
5        5.0  30.50200  114.38800       2       2          48,166;194,313
6        6.0  30.61600  114.39200       1       2          57,121;194,258
7        7.0  30.60800  114.36100       2       3  21,137;145,260;335,453
8        8.0  30.52200  114.33600       1       1                 188,253
9        9.0  30.49800  114.39600       1       2         141,206;369,431
10      10.0  30.48000  114.32100       1       1                 107,168
11      11.0  30.61100  114.38500       3       1                 236,416
Job(cust_no=0, xcoord=30.52211, ycoord

In [ ]:
import gurobipy as gp
from gurobipy import GRB
import json
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime, timedelta


def solve_dsn_scheduling(
    graph, jobs, I, A, T, S, e_iw, l_iw, W, M, W_set, W_set_job, W_job
):
    """
    Solve the DSN scheduling problem using Gurobi.

    Parameters:
    graph (SmartGraph): Graph representing the scheduling problem
    jobs (list): List of Job objects
    I (list): Set of jobs
    A (list): Set of antennas
    T (int): Total time period
    S (list of tuples): Set of synchronous job pairs
    e_iw (dict): Earliest start time for job i in time window w
    l_iw (dict): Latest end time for job i in time window w
    W (dict): Dictionary of time windows for each job i
    M (int): A sufficiently large constant

    Returns:
    dict: Solution containing variable values if optimal solution is found, else None
    """

    # Extract nodes and edges from the graph
    V = graph.nodes
    E = set((u.cust_no, v.cust_no) for u in V for v, _ in graph.get_edges(u))
    s = {u.cust_no: u.duration for u in jobs}
    start = 0
    end = 12
    s[start] = 0
    s[end] = 0

    # remove first and last element from I
    
    s_uv = {
        (u.cust_no, v.cust_no): weight for u in V for v, weight in graph.get_edges(u)
    }

    Ifull = [job.cust_no for job in jobs]
    IplusEnd = I[1:]
    IplusStart = I[:-1]
    I = I[1:-1]
    print(I)

    # Create a new model
    model = gp.Model("DSN_Scheduling")

    # Decision variables
    x = model.addVars(E, A, vtype=GRB.BINARY, name="x")
    z = model.addVars(W_set, vtype=GRB.BINARY, name="z")
    y = model.addVars(W_set_job, vtype=GRB.BINARY, name="y")
    print(y)
    w = model.addVars(Ifull, A, vtype=GRB.CONTINUOUS, name="w")
    model.update()

    # Objective: Maximize the number of successfully scheduled jobs
    model.setObjective(gp.quicksum(x[u, v, a] for (u, v) in E for a in A), GRB.MAXIMIZE)
    print("E", E)
    # Constraints
    # Start Constraint
    model.addConstrs(
        (gp.quicksum(x[start, v, a] for v in IplusEnd if (start, v) in E) == 1 for a in A),
        name="start",
    )
    print(I)

    # Flow Conservation Constraint
    for a in A:
        for u in I:
            model.addConstr(
                gp.quicksum(x[u, v, a] for v in IplusEnd if (u, v) in E)
                == gp.quicksum(x[v, u, a] for v in IplusStart if (v, u) in E),
                name=f"flow_conservation_{u}_{a}",
            )

    # End Constraint
    model.addConstrs(
        (gp.quicksum(x[u, end, a] for u in IplusStart if (u, end) in E) == 1 for a in A),
        name="end",
    )
    
    '''
    # Prevent Duplicate Connections
    for v in I:
        if v not in [start, end]:
            model.addConstrs(
                (gp.quicksum(x[u, v, a] for u in Ifull if (u, v) in E) <= 1  for a in A),
                name=f"duplicate_connection_{v}",
            )
    '''

    # Viewpoint Selection Constraints
    for a in A:
        for v in Ifull:
            if v not in [start, end]:
                exp = 0
                if (a, v) in W:
                    exp = gp.quicksum(z[a, v, w_] for w_ in W[(a, v)])
                model.addConstr(
                    gp.quicksum(x[u, v, a] for u in I if (u, v) in E) == exp,
                    name=f"viewpoint_selection_{v}_{a}",
                )

    for v in Ifull:
        #if v not in [start, end]:
        expr = gp.quicksum(z[w_] for w_ in W_job[v])
        model.addConstr(expr <= 1, name=f"viewpoint_selection_2_{v}")

    # Time Window Selection Constraints
    for a in A:
        for i in Ifull:
            if (a, i) in W:
                model.addConstr(
                    gp.quicksum(y[i, w_] for w_ in W[(a, i)]) <= 1,
                    name=f"time_window_selection_{i}_{a}",
                )
                for w_ in W[(a, i)]:
                    model.addConstr(
                        z[a, i, w_] <= y[i, w_], name=f"time_window_sync_{i}_{a}_{w_}"
                    )

    '''
    # Time Window Synchronization Constraint
    for a in A:
        for u in I:
            if (a, u) in W:
                for w_ in W[(a, u)]:
                    exp = gp.quicksum(x[u, v, a] for v in I if (u, v) in E)
                    model.addConstr(
                        z[a, u, w_] >= y[u, w_] + exp - 1,
                        name=f"time_window_sync_2_{u}_{a}_{w_}",
                    )
    '''

    # Time Continuity Constraint
    for a in A:
        for u, v in E:
            if u not in [start]:
                model.addConstr(
                    w[v, a] >= w[u, a] + s[u] - M * (1 - x[u, v, a]),
                    name=f"time_continuity_{u}_{v}_{a}",
                )
            if u == start:
                model.addConstr(
                    w[v, a] >= w[u, a] + s[u] - M * (1 - x[u, v, a]),
                    name=f"time_continuity_{u}_{v}_{a}",
                )
            if v == end:
                model.addConstr(
                    w[v, a] <= T,
                    name=f"time_continuity_{u}_{v}_{a}",
                )

    # Earliest Start Time Constraint
    for a in A:
        for u in I:
            if u != start and u != end:
                if (a, u) in W:
                    for w_ in W[(a, u)]:
                        model.addConstr(
                            w[u, a] >= e_iw[a, u, w_] - M * (1 - z[a, u, w_]),
                            name=f"earliest_start_{u}_{a}_{w_}",
                        )
            # if u != start and u != end:
            #    model.addConstr(
            #        w[u, a] <= M * gp.quicksum(z[w_] for w_ in W_job[u]),
            #        name=f"latest_start_{u}_{a}_{w_}",
            #    )

    # Latest End Time Constraint
    for a in A:
        for u in I:
            if u != start and u != end:
                if (a, u) in W:
                    for w_ in W[(a, u)]:
                        model.addConstr(
                            w[u, a] + s[u] <= l_iw[a, u, w_] + M * (1 - z[a, u, w_]),
                            name=f"latest_end_{u}_{a}_{w_}",
                        )

    # silence output
    model.setParam("OutputFlag", 0)
    # Optimize model
    model.optimize()

    # Retrieve results
    solution = None
    if model.status == GRB.OPTIMAL:
        solution = {v.varName: v.x for v in model.getVars()}
        print(f"Optimal solution found with objective value: {model.objVal}")
    else:
        print("No optimal solution found.")

    # if solution:
    #    display_solution(solution, graph, jobs, A)

    return solution




def display_solution(solution, graph, jobs, antennas):
    """
    Display the solution in a structured and readable format.

    Parameters:
    solution (dict): Dictionary of variable values from the optimized model
    graph (SmartGraph): Graph representing the scheduling problem
    jobs (list): List of Job objects
    antennas (list): List of antennas
    """

    start = 0

    # Group solutions by antenna
    antenna_solutions = {}
    antenna_times = {}
    for key, value in solution.items():
        if key.startswith("x") and value == 1:
            print(key, value)
            parts = key.split("[")[1].split("]")[0].split(",")
            i = parts[0]
            j = parts[1]
            a = parts[2]
            if a not in antenna_solutions:
                antenna_solutions[a] = []
            if a not in antenna_times:
                antenna_times[a] = []
            antenna_solutions[a].append((i, j))
            antenna_times[a].append(solution[f"w[{j},{a}]"])

    # Display schedule for each antenna
    total_jobs = 0
    print(antenna_solutions)

    print(f"\nTotal jobs: {total_jobs}")



In [5]:
class SmartGraph:
    def __init__(self):
        self.nodes = []
        self.edges = {}

    def add_node(self, node):
        # Check if node already exists based on customer number
        if node.cust_no in [n.cust_no for n in self.nodes]:
            return
        self.nodes.append(node)
        self.edges[node] = []

    def add_edge(self, from_node, to_node, weight):
        self.edges[from_node].append((to_node, weight))

    def get_edges(self, node):
        return self.edges[node]


def create_graph(jobs):
    graph = SmartGraph()

    # Add all jobs to the graph as nodes
    for job in jobs:
        graph.add_node(job)

    mySet = set()

    # Create edges based on time windows and some criteria
    for job1 in jobs:
        if (job1.cust_no == len(jobs) - 1):
            break
        for job2 in jobs:
            if job1.cust_no != job2.cust_no:
                # Iterate through time windows to find transitions between jobs
                for tw1_start, tw1_end in job1.time_windows:
                    for tw2_start, tw2_end in job2.time_windows:
                        if tw2_start >= tw1_start:
                            # Create an edge with transition time as weight
                            transition_time = distance_matrix[job1.cust_no, job2.cust_no]
                            #print("Adding edge from", job1.cust_no, "to", job2.cust_no, "with transition time", transition_time)
                            graph.add_edge(job1, job2, transition_time)
                            mySet.add((job1.cust_no, job2.cust_no))

    return graph, mySet

graph, job_pairs = create_graph(jobs)
print(job_pairs)


{(4, 9), (5, 1), (5, 10), (8, 9), (10, 6), (9, 8), (0, 5), (2, 11), (6, 2), (7, 1), (6, 11), (7, 10), (3, 6), (5, 3), (8, 2), (9, 1), (8, 11), (0, 7), (2, 4), (11, 7), (1, 8), (6, 4), (7, 3), (3, 8), (8, 4), (9, 3), (0, 9), (11, 9), (7, 5), (3, 1), (3, 10), (5, 7), (9, 5), (0, 2), (1, 3), (5, 9), (9, 7), (11, 4), (10, 8), (1, 5), (6, 1), (7, 9), (3, 5), (5, 2), (5, 11), (10, 1), (1, 7), (2, 6), (7, 2), (7, 11), (3, 7), (5, 4), (9, 2), (8, 6), (10, 3), (1, 9), (0, 11), (2, 8), (7, 4), (6, 8), (3, 9), (5, 6), (10, 5), (1, 2), (0, 4), (2, 1), (1, 11), (2, 10), (6, 10), (3, 2), (3, 11), (10, 7), (1, 4), (0, 6), (2, 3), (6, 3), (3, 4), (8, 3), (10, 9), (1, 6), (0, 8), (2, 5), (9, 11), (6, 5), (8, 5), (10, 2), (9, 4), (0, 1), (10, 11), (0, 10), (2, 7), (6, 7), (7, 6), (4, 7), (5, 8), (8, 7), (10, 4), (9, 6), (0, 3), (0, 12), (2, 9), (6, 9), (7, 8)}


In [ ]:
def create_graph_sets(jobs):
    graph, mySet = create_graph(jobs)

    # I: List of jobs' track_ids (cust_no) + start and end
    I = [job.cust_no for job in jobs]

    W = {}
    e_iw = {}
    l_iw = {}
    W_set = set()
    W_set_job = set()
    W_job = {}
    A = [0, 1, 2, 3, 4, 5]  # Antennas

    # Build W, e_iw, l_iw, W_set, and W_job
    for job in jobs:
        W_job[job.cust_no] = set()
        k = 0  # Index for view periods

        for a in A:
            for vp in job.time_windows:
                # get the first part of the tuple
                if (job.cust_no) not in W:
                    W[(job.cust_no)] = []
                W[(job.cust_no)].append(k)

                # Early and late time windows for job and antenna
                e_iw[(job.cust_no, k)] = vp[0]
                l_iw[(job.cust_no, k)] = vp[1]

                # Add to sets
                W_set.add((job.graph, job_pairs = create_graph(jobs)
print(job_pairs)cust_no, k))
                W_job[job.cust_no].add((job.cust_no, k))
                W_set_job.add((job.cust_no, k))
                k += 1

    print("e_iw:", e_iw)
    print("l_iw:", l_iw)

    T = 10000  # Total time period
    S = []  # Set of synchronous job pairs
    M = 10000  # Some constant for scheduling

    # Call to the scheduling solver (assuming it's implemented)

    #solution = solve_dsn_scheduling(
    #    graph, jobs, I, A, T, S, e_iw, l_iw, W, M, W_set, W_set_job, W_job
    #)
    
    return e_iw, l_iw

# Example usage
file_path = 'instance.txt'  # Replace with your file path
jobs = read_jobs_from_instance_file(file_path)
e, l = create_graph_sets(jobs)
#display_solution(solution, graph, jobs, [0, 1, 2, 3, 4, 5])



    CUST NO.   XCOORD.    YCOORD.  DEMAND  TWNUM.                     TWS
0        0.0  30.52211  114.36467       0       1                   0,480
1        1.0  30.47700  114.29200       1       1                 178,243
2        2.0  30.49000  114.41700       1       3  45,105;113,174;188,250
3        3.0  30.47300  114.32000       3       2          29,206;223,400
4        4.0  30.60100  114.35000       2       1                 318,434
5        5.0  30.50200  114.38800       2       2          48,166;194,313
6        6.0  30.61600  114.39200       1       2          57,121;194,258
7        7.0  30.60800  114.36100       2       3  21,137;145,260;335,453
8        8.0  30.52200  114.33600       1       1                 188,253
9        9.0  30.49800  114.39600       1       2         141,206;369,431
10      10.0  30.48000  114.32100       1       1                 107,168
11      11.0  30.61100  114.38500       3       1                 236,416
e_iw: {(0, 0): 0, (0, 1): 0, (0, 2): 0

In [7]:
jobs

[Job(cust_no=0, xcoord=30.52211, ycoord=114.36467, demand=0, time_windows=[(0, 480)]),
 Job(cust_no=1, xcoord=30.477, ycoord=114.292, demand=1, time_windows=[(178, 243)]),
 Job(cust_no=2, xcoord=30.49, ycoord=114.417, demand=1, time_windows=[(45, 105), (113, 174), (188, 250)]),
 Job(cust_no=3, xcoord=30.473, ycoord=114.32, demand=3, time_windows=[(29, 206), (223, 400)]),
 Job(cust_no=4, xcoord=30.601, ycoord=114.35, demand=2, time_windows=[(318, 434)]),
 Job(cust_no=5, xcoord=30.502, ycoord=114.388, demand=2, time_windows=[(48, 166), (194, 313)]),
 Job(cust_no=6, xcoord=30.616, ycoord=114.392, demand=1, time_windows=[(57, 121), (194, 258)]),
 Job(cust_no=7, xcoord=30.608, ycoord=114.361, demand=2, time_windows=[(21, 137), (145, 260), (335, 453)]),
 Job(cust_no=8, xcoord=30.522, ycoord=114.336, demand=1, time_windows=[(188, 253)]),
 Job(cust_no=9, xcoord=30.498, ycoord=114.396, demand=1, time_windows=[(141, 206), (369, 431)]),
 Job(cust_no=10, xcoord=30.48, ycoord=114.321, demand=1, tim

In [8]:
import gurobipy as gp
from gurobipy import GRB

# Sample input data (adjust according to your problem instance)
V = [job.cust_no for job in jobs]  # List of customers
K = [1, 2, 3]  # List of vehicles
W = [1, 2]  # List of time windows
d = [job.demand for job in jobs]  # Demand for each customer
Q = 100  # Vehicle capacity
T = 1000  # Max time duration
c = {(i, j, k): distance_matrix[i][j] for i in V for j in V for k in K}  # Travel cost
s = {u: 1 for u in V}  # Service time for each customer

end_node = 12

# Earliest and latest times for each customer and time window
#e = {(1, 1): 5, (1, 2): 10, (2, 1): 8, (2, 2): 12, (3, 1): 15, (3, 2): 18, (4, 1): 20, (4, 2): 25}
#l = {(1, 1): 20, (1, 2): 25, (2, 1): 22, (2, 2): 27, (3, 1): 30, (3, 2): 35, (4, 1): 40, (4, 2): 45}

M = 1000  # A large number for big-M constraints
depot = 0

# Create a new model
model = gp.Model("VRP")

# Decision variables
x = model.addVars(V, V, K, vtype=GRB.BINARY, name="x")  # Binary for edges
z = model.addVars(V, W, K, vtype=GRB.BINARY, name="z")  # Binary for time windows
u_var = model.addVars(V, K, vtype=GRB.CONTINUOUS, name="u_var")  # Continuous for time
y = model.addVars(V, W, vtype=GRB.BINARY, name="y")  # Binary for visiting customers

# Objective function (1-1): Minimize the total travel cost
model.setObjective(
    gp.quicksum(c[u, v, k] * x[u, v, k] for u in V for v in V for k in K),
    GRB.MINIMIZE
)

# (1-2)
#model.addConstrs(
#    (gp.quicksum(d[u] * z[u, w, k] for w in W for k in K) >= d[u] for u in V if u != depot),
#    name="demand_satisfaction"
#)

# (1-3)
#model.addConstrs(
#    (gp.quicksum(z[u, w, k] for w in W for k in K) >= 1 for u in V if u != depot),
#    name="min_visits"
#)

# (1-4)
model.addConstrs(
    (gp.quicksum(x[depot, v, k] for v in V if v != depot) == 1 for k in K),
    name="depot_outflow"
)

model.addConstrs(
    (gp.quicksum(x[depot, v, k] for k in K) <= 1 for v in V if v != depot),
    name="unique_depot_departure"
)


# (1-5)
model.addConstrs(
    (gp.quicksum(x[u, v, k] for v in V if v != u and v != depot) ==
     gp.quicksum(x[v, u, k] for v in V if v != u and v != end_node) for u in V if u != depot and u != end_node for k in K),
    name="flow_conservation"
)


# (1-5b)
model.addConstrs(
    (gp.quicksum(x[u, v, k] for v in V) <= 1 for u in V if u != depot and u != end_node for k in K),
    name="rhs_constraint"
)



# (1-6)
model.addConstrs(
    (gp.quicksum(x[u, end_node, k] for u in V if u != end_node and u != depot) == 1 for k in K),
    name="depot_inflow"
)


# (1-7)
model.addConstrs(
    (gp.quicksum(x[u, v, k] for k in K for u in V if u != v and u != depot and u != end_node) <= 1 for v in V if v != depot and v != end_node),
    name="no_duplicate_visits"
)

# visit all nodes
model.addConstrs(
    (gp.quicksum(x[u, v, k] for k in K for u in V if u != v and u != depot and u != end_node) == 1 for v in V if v != depot and v != end_node),
    name="visit_all_nodes"
)



# (1-8)
model.addConstrs(
    (gp.quicksum(x[u, v, k] for v in V if v != u and v != end_node) ==
     gp.quicksum(z[u, w, k] for w in W) for u in V for k in K if u != depot),
    name="flow_time_window_sync"
)


# (1-8b)
model.addConstrs(
    (gp.quicksum(z[u, w, k] for w in W) <= 1 for u in V for k in K if u != depot and u != end_node),
    name="time_window_selection"
)

# (1-9)
#model.addConstrs(
#    (gp.quicksum(d[u] * z[u, w, k] for u in V if u != depot for w in W) <= Q for k in K),
#    name="capacity"
#)

# (1-10)
model.addConstrs(
    (gp.quicksum(y[u, w] for w in W) == 1 for u in V for k in K if u != depot),
    name="time_window_selection"
)

# (1-11)
model.addConstrs(
    (z[u, w, k] <= y[u,w] for u in V if u != depot for w in W for k in K),
    name="time_sync"
)

# (1-12)
#model.addConstrs(
#    (z[u, w, k] >= y[u,w] + gp.quicksum(x[u, v, k] for v in V) - 1 for u in V if u != depot for w in W for k in K),
#    name="time_sync_2"
#)

# (1-13)
#model.addConstrs(
#    (u_var[v, k] >= u_var[u, k] + s[u] + c[u, v, k] - M * (x[u, v, k] - 1)
#     for u in V for v in V for k in K if u != v),
#    name="time_continuity"
#)

# (1-14)
model.addConstrs(
    (u_var[u, k] >= e[u, w] + M * (z[u, w, k] - 1)
     for u in V if u != depot for k in K for w in W if (u, w) in e),
    name="earliest_start"
)

# (1-15)
model.addConstrs(
    (u_var[u, k] + s[u] <= l[u, w] + M * (1 - z[u, w, k])
     for u in V if u != depot for k in K for w in W if (u, w) in l),
    name="latest_end"
)


# Solve the model
model.optimize()

x_dict = {}
# Display solution
if model.status == GRB.OPTIMAL:
    # print all the variables
    for v in model.getVars():
        if v.x > 0:
            if v.varName.startswith("x"):
                x_dict[v.varName] = v.x
            print(f"{v.varName}: {v.x}")

from collections import defaultdict

# Function to process x_dict and print the route for each vehicle
def process_routes(x_dict):
    vehicle_routes = defaultdict(list)

    # Extract routes for each vehicle from x_dict
    for key in x_dict:
        # Example variable name: "x[0,5,2]" -> split into from_node, to_node, vehicle
        key_parts = key.strip("x[]").split(",")
        from_node = int(key_parts[0])
        to_node = int(key_parts[1])
        vehicle = int(key_parts[2])
        
        # Store the arc (from_node, to_node) for the corresponding vehicle
        vehicle_routes[vehicle].append((from_node, to_node))
    
    # Reconstruct and print the routes for each vehicle
    for vehicle, arcs in vehicle_routes.items():
        # Find the starting node (depot) and trace the route
        route = [0]  # Assuming depot is node 0
        while True:
            # Find the arc that starts from the last node in the route
            next_arc = next((arc for arc in arcs if arc[0] == route[-1]), None)
            if next_arc:
                route.append(next_arc[1])  # Add the next node to the route
                arcs.remove(next_arc)      # Remove the arc once used
            else:
                break
        
        # Print the reconstructed route for the vehicle
        print(f"Vehicle {vehicle} route: {' -> '.join(map(str, route))}")

# Process and print the routes
process_routes(x_dict)


Set parameter Username
Academic license - for non-commercial use only - expires 2025-01-17
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Gentoo Linux")

CPU model: Intel(R) Core(TM) i9-14900K, instruction set [SSE2|AVX|AVX2]
Thread count: 32 physical cores, 32 logical processors, using up to 32 threads

Optimize a model with 427 rows, 650 columns and 2961 nonzeros
Model fingerprint: 0x46df57ec
Variable types: 39 continuous, 611 integer (611 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [7e-03, 2e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Presolve removed 201 rows and 144 columns
Presolve time: 0.00s
Presolved: 226 rows, 506 columns, 2178 nonzeros
Variable types: 0 continuous, 506 integer (506 binary)
Found heuristic solution: objective 0.5128513

Root relaxation: objective 4.045394e-01, 62 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Wor